In [ ]:
# Cell for hyperparameter tuning
import sys
sys.path.append('..')
from scripts.tune_hyperparams import run_optimization

# Run 50 trials of hyperparameter optimization
data_path = '../data/raw/eurusd_1y.csv'
best_params = run_optimization(data_path, 50)
print(f"Best hyperparameters found: {best_params}")

## 3. Define Hyperparameter Search Spaces

Display the search spaces defined in hyperparam_config.py.

In [ ]:
print("Hyperparameter Search Spaces:")
for param, config in HYPERPARAM_SEARCH_SPACE.items():
    print(f"{param}: {config}")

print("\nDefault PPO Parameters:")
for param, value in DEFAULT_PPO_PARAMS.items():
    print(f"{param}: {value}")

print("\nExample suggested params:")
# Mock trial for demonstration
class MockTrial:
    def suggest_float(self, name, low, high, log=False):
        return (low + high) / 2
    def suggest_int(self, name, low, high):
        return (low + high) // 2
    def suggest_categorical(self, name, choices):
        return choices[len(choices)//2]

mock_trial = MockTrial()
example_params = suggest_hyperparams(mock_trial)
for param, value in example_params.items():
    print(f"{param}: {value}")

In [ ]:
# Load historical data from Drive
try:
    data = load_historical_data('/content/drive/MyDrive/data/raw/eurusd_1y.csv')
    print(f"Loaded {len(data)} data points from Drive")
except Exception as e:
    print(f"Could not load data from Drive: {e}")
    data = None

env = HistoricalTradingEnv(data, asset='EURUSD')

## 4. Set Up Optimization Function

The objective function is defined in hyperparam_optimizer.py. It trains the PPO model with suggested hyperparameters and returns the average reward.

In [ ]:
# The objective function is imported from src.models.hyperparam_optimizer
# It suggests hyperparameters, trains PPO, and returns average reward
print("Objective function ready for optimization.")

## 5. Run Hyperparameter Tuning

Execute the optimization process using Optuna.

In [ ]:
# Run hyperparameter optimization
n_trials = 50  # Colab GPU can handle 200+ trials
best_params, study = run_optimization(env, n_trials=n_trials, study_name='ppo_trading_bot_colab')

print("Optimization complete!")
print("Best parameters:", best_params)

In [ ]:
# Visualize optimization results
import optuna.visualization as vis

# Parameter importance
fig = vis.plot_param_importances(study)
fig.show()

# Optimization history
fig2 = vis.plot_optimization_history(study)
fig2.show()

In [ ]:
# Save best parameters to Drive
import json

with open('/content/drive/MyDrive/best_hyperparams.json', 'w') as f:
    json.dump(best_params, f, indent=4)

print("Best hyperparameters saved to Drive!")

## 6. Integrate Tuned Parameters into Model

Use the best hyperparameters for training the final model.

In [ ]:
# Create PPO model with best parameters
model = PPO('MlpPolicy', env, **best_params, verbose=1)

# Train the model with full timesteps
from src.utils.hyperparam_config import FULL_TRAIN_TIMESTEPS
model.learn(total_timesteps=FULL_TRAIN_TIMESTEPS)

# Save the tuned model to Drive
model.save('/content/drive/MyDrive/data/models/tuned_ppo_model.zip')
print("Tuned model saved to Drive!")

## GPU Optimization Tip

Optuna automatically uses CUDA if available. For best results with 1-year simulation data:

- Set `n_trials = 200` for comprehensive optimization
- Use Colab Pro GPU runtime (~2-3 hours)
- Monitor GPU usage in Colab's runtime info
- Results will be saved to your Google Drive

## 7. Validate Tuning Results

Run tests to ensure the tuned model performs well.

In [ ]:
# Run validation tests
import subprocess
result = subprocess.run(['python', '-m', 'pytest', 'tests/test_hyperparam_optimizer.py', '-v'], capture_output=True, text=True)
print("Test Output:")
print(result.stdout)
if result.stderr:
    print("Errors:")
    print(result.stderr)